In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd 
import json

In [2]:
def get_data(link,c):
    try:
        x=requests.get(link)
        soup=bs(x.text,'html5lib')
        title=soup.find(class_="product-meta__title heading h1").text
        price=int("".join([i for i in soup.find(class_="price").text if i.isdigit()]))
        Game_Information=soup.find(class_="rte text--pull").find("ul")
        Game_Information={i.find("h5").text:i.find('p').text for i in Game_Information.find_all('li')}
        img_link='https:'+soup.find(class_='card__section card__section--tight').find('noscript').find('img')['src']
        img= requests.get(img_link)
        informations={
        "id":f"a{c}",
        "cat":link[36:44],
        "title":f"{title}",
        "price":f"{price}",
        "Game_Information":Game_Information,
        "Description":soup.find_all(class_='card')[-1].find("p").text,
        'img':img
        }

        with open(f'./imgs/a{c}.png','wb') as f :
            f.write(img.content)
        return informations
    except:
        return f" oopssss  {link}"


In [3]:
def links_ex(link):
    x=requests.get(link)
    soup=bs(x.text,'html5lib')
    links=soup.find_all('a',class_="product-item__image-wrapper")
    links=['https://shamystores.com'+i['href'] for i in links]
    return links

all_links=links_ex('https://shamystores.com/collections/ps5-games')
all_links.extend(links_ex('https://shamystores.com/collections/ps5-games?page=2'))
all_links.extend(links_ex('https://shamystores.com/collections/ps4-games'))
all_links.extend(links_ex('https://shamystores.com/collections/ps4-games?page=2'))
all_links.extend(links_ex('https://shamystores.com/collections/xbox-games'))
all_links.extend(links_ex('https://shamystores.com/collections/xbox-games?page=2'))
all_links.extend(links_ex('https://shamystores.com/collections/nintendo-switch-games'))
all_links.extend(links_ex('https://shamystores.com/collections/nintendo-switch-games?page=2'))
len(all_links)

190

In [4]:
all_data=[]
c= 0
for link in all_links:
    x=get_data(link,c)
    if "oopssss" not in x:
        c+=1
        all_data.append(x)

In [5]:
import pickle
with open('arr.pkl', "wb") as f:
    pickle.dump(all_data, f)

In [6]:
df=pd.DataFrame ([i for i in  all_data if "oopssss" not in i])

In [31]:
len(all_data)

134

In [7]:
df['Game_Information']=df['Game_Information'].apply(lambda x: json.dumps(x))

In [32]:
from random import randint

In [11]:
df['rate']=0
df['rate']=df['rate'].apply(lambda x:randint(0, 5))

In [36]:
import sqlite3 as sql

In [37]:
import sqlite3 as sql
conn = sql.connect('data__db.db')
conn

In [39]:
df.to_sql('users', con=conn)

134

In [40]:
column_types = dict(df.dtypes)
column_types

{'id': dtype('O'),
 'cat': dtype('O'),
 'title': dtype('O'),
 'price': dtype('O'),
 'Game_Information': dtype('O'),
 'Description': dtype('O'),
 'img': dtype('O'),
 'rate': dtype('int64')}

In [41]:
column_types={}
for i , v in df.dtypes.items():
    column_types[i]=v


In [42]:
column_types = dict(df.dtypes)

columns = []

for column_name in column_types:
    
    if column_types[column_name] == 'object':
        dtype = 'text'
    elif column_types[column_name] == 'int64':
        dtype = 'integer'
    elif column_types[column_name] == 'int32':
        dtype = 'integer'
        
    elif column_types[column_name] == 'float64':
        dtype = 'real'
    else:
        print(column_types[column_name])
        raise Exception
        
    columns.append(f'{column_name} {dtype}')

s = ',\n'.join(columns)
print(s)

id text,
cat text,
title text,
price text,
Game_Information text,
Description text,
img text,
rate integer


In [43]:
create_table = '''
                    create table products(
                    id text,
                    cat text,
                    title text,
                    price text,
                    Game_Information text,
                    Description text,
                    img blob,
                    rate integer
                    );
               '''

In [44]:
conn.execute(create_table)